<a href="https://colab.research.google.com/github/stic-lab/AG-037/blob/main/MAGAN_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function

# import os
from os import listdir
import numpy as np
from numpy import asarray
from numpy.random import randn
from tensorflow.keras.optimizers import Adam
# import the necessary packages
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.data import AUTOTUNE
from keras.losses import BinaryCrossentropy, MeanSquaredError
from keras.layers import Activation, Concatenate,Input, Dense, Reshape, Flatten, Dropout, LeakyReLU, RandomCrop
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, AveragePooling2D, UpSampling2D, Conv2DTranspose
from keras.models import Model, Input,Sequential
from keras.preprocessing.image import img_to_array, load_img
# import tensorflow as tf
from tensorflow import sigmoid, reduce_max, nn, concat, image, shape
from matplotlib import pyplot as plt
import tensorflow as tf

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
image_shape =(256,256,3) #(286, 286)
batch_size = 64
epochs = 200
save_dir = "./"
path = "/content/drive/MyDrive/DATA/train/"

### Some useful function that could be used to load data

In [ ]:
trainAug = Sequential([
	preprocessing.Rescaling(scale=1.0 / 255),
	preprocessing.RandomFlip("horizontal_and_vertical"),
	preprocessing.RandomZoom(
		height_factor=(-0.05, -0.15),
		width_factor=(-0.05, -0.15)),
	preprocessing.RandomRotation(0.3)
])

def load_images(path, size=image_shape):
  data_list = list()
  # enumerate filenames in directory, assume all are images
  for filename in listdir(path):
    # load and resize the image
    pixels = load_img(path + filename, target_size=size)
    # convert to numpy array
    pixels = img_to_array(pixels)
    # store
    data_list.append(pixels)

  datas = asarray(data_list)
  datas = datas.astype('float32') / 255.0
  dataset = tf.data.Dataset.from_tensor_slices(datas)
  # dataset = dataset.batch(128)
  dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)
  return dataset

In [ ]:
# load dataset (low light image)
dataL_all = load_images(path + 'low/')
dataG_all = (
	dataL_all
	.shuffle(batch_size * 100)
	# .batch(batch_size)
	.map(lambda x: (trainAug(x)),
		 num_parallel_calls=tf.data.AUTOTUNE)
	.prefetch(tf.data.AUTOTUNE)
)
dataL_all = (
	dataL_all
	.shuffle(batch_size * 100)
	# .batch(batch_size)
	.map(lambda x: (trainAug(x)),
		 num_parallel_calls=tf.data.AUTOTUNE)
	.prefetch(tf.data.AUTOTUNE)
)
# load dataset (normal light image)
dataH_all = load_images(path + 'normal/')
dataH_all = (
	dataH_all
	.shuffle(batch_size * 100)
	# .batch(batch_size)
	.map(lambda x: (trainAug(x)),
		 num_parallel_calls=tf.data.AUTOTUNE)
	.prefetch(tf.data.AUTOTUNE)
)
# Prepare the training dataset.
# train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

Visualize some samples


In [ ]:
#Displaying images
# dataL = dataL_all.take(4)
# dataH = dataH_all.take(4)
# _, ax = plt.subplots(4, 2, figsize=(10, 15))
# print(dataL)
# # plt.figure(figsize=(20, 4))
# steps = tf.data.Dataset.range(4)
# # steps = range(0,4)
# i=0;
# for L, H in enumerate(zip(dataL, dataH)):
#     ax[i, 0].imshow(L)
#     ax[i, 1].imshow(H)
#     i=i+1
# plt.show()


# _, ax = plt.subplots(4, 2, figsize=(10, 15))
# for i, samples in enumerate(zip(dataL_all.take(4), dataH_all.take(4))):
#     low = ((samples[0][0] * 255.0).numpy()).astype(np.uint8)
#     normal = ((samples[1][0] * 255.0).numpy()).astype(np.uint8)
#     ax[i, 0].imshow(low)
#     ax[i, 1].imshow(normal)
# plt.show()

In [ ]:
discriminator_local = Sequential(
    [
        Input(shape=image_shape),
        RandomCrop(28, 28),
        Conv2D(8, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(alpha=0.2),
        GlobalMaxPooling2D(),
        Dense(1),
    ],
    name="local_discriminator",
)
discriminator_local.summary()

Model: "local_discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_crop_3 (RandomCrop)  (None, 28, 28, 3)         0         
                                                                 
 conv2d_36 (Conv2D)          (None, 14, 14, 8)         224       
                                                                 
 leaky_re_lu_240 (LeakyReLU)  (None, 14, 14, 8)        0         
                                                                 
 conv2d_37 (Conv2D)          (None, 7, 7, 16)          1168      
                                                                 
 leaky_re_lu_241 (LeakyReLU)  (None, 7, 7, 16)         0         
                                                                 
 conv2d_38 (Conv2D)          (None, 4, 4, 32)          4640      
                                                                 
 leaky_re_lu_242 (LeakyReLU)  (None, 4, 4, 32) 

In [ ]:
discriminator_global = Sequential(
    [
        Input(shape=image_shape),
        Conv2D(8, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(alpha=0.2),
        GlobalMaxPooling2D(),
        Dense(1),
    ],
    name="discriminator_global",
)
discriminator_global.summary()

Model: "discriminator_global"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 leaky_re_lu_246 (LeakyReLU)  (None, 128, 128, 8)      0         
                                                                 
 conv2d_43 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 leaky_re_lu_247 (LeakyReLU)  (None, 64, 64, 16)       0         
                                                                 
 conv2d_44 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 leaky_re_lu_248 (LeakyReLU)  (None, 32, 32, 32)       0         
                                                                 
 conv2d_45 (Conv2D)          (None, 16, 16, 64

Feature attention function

In [ ]:
def featureAttention(input_feature, idx):
  
    #Maxpoolin conv sigmoid conv
    fa_max = MaxPooling2D(name="fa_max"+idx, pool_size=(input_feature[0].shape[0], input_feature[0].shape[1]), strides=1, padding='valid')(input_feature)
    fa_conv_max = Conv2D(name="fa_conv_max"+idx,filters=input_feature[0].shape[2],kernel_size=3, padding='same', activation=LeakyReLU())(fa_max)
    fa_sigmoid_max = sigmoid(fa_conv_max)
    fa_conv_sigmoid_max = Conv2D(name="fa_conv_sigmoid_max"+idx, filters=input_feature[0].shape[2],kernel_size=3, padding='same', activation=LeakyReLU())(fa_sigmoid_max)
    #Average conv sigmoid conv
    fa_avg = AveragePooling2D(name="fa_avg"+idx, pool_size=(input_feature[0].shape[0], input_feature[1].shape[1]), strides=1, padding='valid')(input_feature)
    fa_conv_avg = Conv2D(name="fa_conv_avg"+idx, filters=input_feature[0].shape[2],kernel_size=3, padding='same', activation=LeakyReLU())(fa_avg)
    fa_sigmoid_avg = sigmoid(fa_conv_avg)
    fa_conv_sigmoid_avg = Conv2D(name="fa_conv_sigmoid_avg"+idx, filters=input_feature[0].shape[2],kernel_size=3, padding='same', activation=LeakyReLU())(fa_sigmoid_avg)
    #add the result of the two last conv from the max and avg pool and perform sigmoid on it
    sum_conv_max_avg = sigmoid(fa_conv_sigmoid_max+fa_conv_sigmoid_avg)
    return sum_conv_max_avg

Pixel attention function

In [ ]:
def mixedAttention(input_feature, idx):
  featureAtt = featureAttention(input_feature,idx)
  intermediate = input_feature * featureAtt
  #Maxpoolin conv sigmoid conv
  pa_max = MaxPooling2D(name="pa_max"+idx, pool_size=(1,input_feature[0].shape[2]), strides=1, padding='same')(intermediate)
  pa_conv_max = Conv2D(name="pa_conv_max"+idx, filters=1, kernel_size=1, padding='valid', activation=LeakyReLU())(pa_max)
    #Average conv sigmoid conv
  pa_avg = AveragePooling2D(name="pa_avg"+idx, pool_size=(1, input_feature[0].shape[2]), strides=1, padding='same')(intermediate)
  pa_conv_avg = Conv2D(name="pa_conv_avg"+idx, filters=1, kernel_size=1, padding='valid', activation=LeakyReLU())(pa_avg)
    #Concatenation
  pa_concat = concat(name="pa_conv_concat"+idx, values=[pa_conv_max,pa_conv_avg], axis=3)
    
  pa_conv_concat = Conv2D(name="pa_conv_concat"+idx, filters=input_feature[0].shape[2], kernel_size=3, padding='same', activation=LeakyReLU())(pa_concat)
  #add the result of the two last conv from the max and avg pool and perform sigmoid on it
  sum_conv_max_concat = sigmoid(pa_conv_concat)
  pixelAtt = input_feature * sum_conv_max_concat
  return pixelAtt

Define the Generative part (Auto encoder network)

In [ ]:
def Generator():
  
    ###############*******************************#################
    inputs = Input(name="input_image", shape=image_shape)
    conv01 = Conv2D(name="conv01", filters=8,kernel_size=16, padding='same', activation=LeakyReLU(alpha=0.2))(inputs)
    ##################################
    #Start the first feature attention
    feat0 = featureAttention(input_feature=conv01, idx="0")
    out_feature0 = feat0*conv01
    # End feature attention
    ##################################
    conv02 = Conv2D(name="conv02", filters=8,kernel_size=3, padding='same', activation=LeakyReLU())(out_feature0)
    max_pool01 = MaxPooling2D(name="max_pool01", pool_size=(3, 3), strides=2, padding='same')(conv02)
    ###############*******************************#################
    
    ###############*******************************#################
    conv03 = Conv2D(name="conv03", filters=16,kernel_size=3, padding='same', activation=LeakyReLU())(max_pool01)
    conv04 = Conv2D(name="conv04", filters=16,kernel_size=3, padding='same', activation=LeakyReLU())(conv03)
    ##################################
    mixed_att_01 = mixedAttention(input_feature=conv04, idx="1")
    ##################################
    max_pool02 = MaxPooling2D(name="max_pool02", pool_size=(3, 3), strides=2, padding='same')(mixed_att_01)
    ###############*******************************#################

    
    ###############*******************************#################
    conv05 = Conv2D(name="conv05", filters=32,kernel_size=3, padding='same', activation=LeakyReLU())(max_pool02)
    conv06 = Conv2D(name="conv06", filters=32,kernel_size=3, padding='same', activation=LeakyReLU())(conv05)
    mixed_att_02 = mixedAttention(input_feature=conv06, idx="2")
    max_pool03 = MaxPooling2D(name="max_pool03", pool_size=(3, 3), strides=2, padding='same')(mixed_att_02)
    ###############*******************************#################

    ###############*******************************#################
    conv07 = Conv2D(name="conv07", filters=64,kernel_size=3, padding='same', activation=LeakyReLU())(max_pool03)
    conv08 = Conv2D(name="conv08", filters=64,kernel_size=3, padding='same', activation=LeakyReLU())(conv07)
    mixed_att_03 = mixedAttention(input_feature=conv08, idx="3")
    max_pool04 = MaxPooling2D(name="max_pool04", pool_size=(3, 3), strides=2, padding='same')(mixed_att_03)
    ###############*******************************#################
    
    
    ###############*******************************#################
    conv09 = Conv2D(name="conv09", filters=128,kernel_size=3, padding='same', activation=LeakyReLU())(max_pool04)
    conv10 = Conv2D(name="conv10", filters=128,kernel_size=3, padding='same', activation=LeakyReLU())(conv09)
        ##################################
    mixed_att_04 = mixedAttention(input_feature=conv10, idx="4")
    ##################################
    ###############*******************************#################
    




    ###############*******************************#################
    conv11 = Conv2D(name="conv11", filters=128,kernel_size=3, padding='same', activation=LeakyReLU())(mixed_att_04)
    conv12 = Conv2D(name="conv12", filters=128,kernel_size=3, padding='same', activation=LeakyReLU())(conv11)
    concat01 = tf.keras.layers.Concatenate(name="concatenate01", axis=3)([mixed_att_04, conv12])
    upsampling1 = UpSampling2D(name="upsampling1", size=(2, 2))(concat01)
    ###############*******************************#################
    
    ###############*******************************#################
    conv13 = Conv2D(name="conv13", filters=64,kernel_size=16, padding='same', activation=LeakyReLU())(upsampling1)
    ##################################
    #Start the first feature attention
    feat5 = featureAttention(input_feature=conv13, idx="5")
    out_feature5 = feat5*conv13
    # End feature attention
    ##################################
    conv14 = Conv2D(name="conv14", filters=64,kernel_size=3, padding='same', activation=LeakyReLU())(out_feature5)
    concat02 = tf.keras.layers.Concatenate(name="concatenate02", axis=3)([mixed_att_03, conv14])
    upsampling2 = UpSampling2D(name="upsampling2", size=(2, 2))(concat02)
    ###############*******************************#################
    
    ###############*******************************#################
    conv15 = Conv2D(name="conv15", filters=32,kernel_size=16, padding='same', activation=LeakyReLU())(upsampling2)
    ##################################
    #Start the first feature attention
    feat6 = featureAttention(input_feature=conv15, idx="6")
    out_feature6 = feat6*conv15
    # End feature attention
    ##################################
    conv16 = Conv2D(name="conv16", filters=32,kernel_size=3, padding='same', activation=LeakyReLU())(out_feature6)
    concat03 = tf.keras.layers.Concatenate(name="concatenate03", axis=3)([mixed_att_02, conv16])
    upsampling3 = UpSampling2D(name="upsampling3", size=(2, 2))(concat03)
    ###############*******************************#################

    
    ###############*******************************#################
    conv17 = Conv2D(name="conv17", filters=16,kernel_size=16, padding='same', activation=LeakyReLU())(upsampling3)
    ##################################
    #Start the first feature attention
    feat7 = featureAttention(input_feature=conv17, idx="7")
    out_feature7 = feat7*conv17
    # End feature attention
    ##################################
    conv18 = Conv2D(name="conv18", filters=16,kernel_size=3, padding='same', activation=LeakyReLU())(out_feature7)
    concat04 = tf.keras.layers.Concatenate(name="concatenate04", axis=3)([mixed_att_01, conv18])
    upsampling4 = UpSampling2D(name="upsampling4", size=(2, 2))(concat04)
    ###############*******************************#################

    
    ###############*******************************#################
    conv19 = Conv2D(name="conv19", filters=8,kernel_size=16, padding='same', activation=LeakyReLU())(upsampling4)
    ##################################
    #Start the first feature attention
    feat8 = featureAttention(input_feature=conv19, idx="8")
    out_feature8 = feat8*conv19
    # End feature attention
    ##################################
    conv20 = Conv2D(name="conv20", filters=8,kernel_size=3, padding='same', activation=LeakyReLU())(out_feature8)
    concat01 = tf.keras.layers.Concatenate(axis=3)([conv02, conv20])

    conv21 = Conv2D(name="conv21", filters=3,kernel_size=3, padding='same')(concat01)
    ###############*******************************#################
    output = tf.keras.layers.Add()([inputs, conv21])


    model = Model(inputs, output)
    # opt = Adam(lr=0.0002, beta_1=0.5)
    model._name="Generator_auto_encoder"
    # model.compile(loss=['mae', 'mae'], optimizer=opt, metrics=['accuracy'])
    return model

generator = Generator()
print(generator.summary())

Model: "Generator_auto_encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv01 (Conv2D)                (None, 256, 256, 8)  6152        ['input_image[0][0]']            
                                                                                                  
 fa_max0 (MaxPooling2D)         (None, 1, 1, 8)      0           ['conv01[0][0]']                 
                                                                                                  
 fa_avg0 (AveragePooling2D)     (None, 1, 1, 8)      0           ['conv01[0][

In [ ]:
def gram_matrix(input_tensor):
  result = tf.linag.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  gram_matrix = tf.expand_dims(result, axis=0)
  input_shape = tf.shape(input_tensor)
  i_j = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return gram_matrix/i_j

In [ ]:
def load_vgg():
  vgg = tf.keras.applications.VGG9(include_top=True, weights="imagenet")
  vgg.trainable = False
  content_layers = ['block4_conv2']
  style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
  content_output = vgg.get_layer(content_layers[0]).output
  style_output = [vgg.get_layer(style_layer).output for style_layer in style_layers]
  gram_style_output = [gram_matrix(output_) for output_ in style_output]
  
  model = Model([vgg.input], [content_output, gram_style_output])
  return model

In [ ]:
def loss_object(style_outputs, content_outputs, style_target, content_target):
  style_weight = 1e-2
  content_weight = 1e-4
  content_loss = tf.reduce_mean((content_outputs - content_target)**2)
  style_loss = tf.add_n(tf.reduce_mean((output_ - target_)**2) for output_, target_ in zip(style_outputs, style_target))
  total_loss = style_weight*style_loss+content_weight*content_loss

In [ ]:
vgg_model = load_vgg()

In [ ]:
# Instantiate one optimizer for the discriminator and another for the generator.
d_optimizer = Adam(learning_rate=0.0003)
g_optimizer = Adam(learning_rate=0.0004)

# Instantiate a loss function.
loss_fn = MeanSquaredError()


@tf.function
def train_step(low, high, for_generator):
  
    #Select a random half batch of normal light image
    # idx = np.random.randint(0, dataH_all.shape[0], int(batch_size/2))
    # imgs_N = dataH_all[idx]
    idxs = tf.range(tf.shape(high)[0])
    ridxs = tf.random.shuffle(idxs)[:high.shape[0]]
    imgs_N = tf.gather(high, ridxs)
    #Select a random half batch of low light image
    # idx = np.random.randint(0, dataL_all.shape[0], int(batch_size/2))
    # imgs_L = dataL_all[idx]
    idxs = tf.range(tf.shape(low)[0])
    ridxs = tf.random.shuffle(idxs)[:low.shape[0]]
    imgs_L = tf.gather(low, ridxs)

    # Decode them to fake images
    generated_images = generator(imgs_L)
    # Combine them with real images
    combined_images = tf.concat([generated_images, imgs_N], axis=0)

    # Assemble labels discriminating real from fake images
    # labels = tf.concat(
    #     [tf.zeros((generated_images.shape[0], 1)), tf.ones((imgs_N.shape[0], 1))], axis=0
    # )
    labels = tf.concat(
        [tf.zeros((generated_images.shape[0], 1)), tf.ones((imgs_N.shape[0], 1))], axis=0
    )
    # Add random noise to the labels - important trick!
    labels += 0.05 * tf.random.uniform(labels.shape)

    # Train the global discriminator
    with tf.GradientTape() as tape:
        predictions = discriminator_global(combined_images)
        d_loss_global = loss_fn(labels, predictions)
    grads = tape.gradient(d_loss_global, discriminator_global.trainable_weights)
    d_optimizer.apply_gradients(zip(grads, discriminator_global.trainable_weights))
    # Train the local discriminator
    with tf.GradientTape() as tape:
        predictions = discriminator_local(combined_images)
        d_loss_local = loss_fn(labels, predictions)
    grads = tape.gradient(d_loss_local, discriminator_local.trainable_weights)
    d_optimizer.apply_gradients(zip(grads, discriminator_local.trainable_weights))

    #Select a random half batch of low light image
    # idx = np.random.randint(0, dataL_all.shape[0], batch_size)
    # imgs_L = dataL_all[idx]
    imgs = low
    # Assemble labels that say "all real images"
    misleading_labels = tf.ones((imgs.shape[0], 1))

    # Train the generator (note that we should *not* update the weights
    # of the discriminator)!
    with tf.GradientTape() as tape:
        g_img = generator(imgs)
        predictions_global = discriminator_global(g_img)
        g_loss_global = loss_fn(misleading_labels, predictions_global)
        predictions_local = discriminator_global(g_img)
        g_loss_local = loss_fn(misleading_labels, predictions_local)
        # Content and style loss
        output_target = vgg_model(g_img*255)
        output_generated = vgg_model(high*255)
        content_style_loss = loss_object(output_generated[1], output_generated[0],output_target[1], output_target[0])

        g_loss = g_loss_global + g_loss_local + content_style_loss
    grads = tape.gradient(g_loss, generator.trainable_weights)
    g_optimizer.apply_gradients(zip(grads, generator.trainable_weights))

    return d_loss_global, d_loss_local, g_loss,  generated_images

In [ ]:
def train(epochs, save_interval=50):
  
  print("Start training")
  for epoch in range(epochs):
    # Train the discriminator & generator on one batch of real images.
    for step, (x_batch_train, y_batch_train) in enumerate(zip(dataL_all, dataH_all)):
      d_loss_global, d_loss_local, g_loss, generated_images = train_step(x_batch_train, y_batch_train, for_generator)
      if step % 2 == 0:
        # Print metrics
        print("epoch> %d | step> %d | D global>%.2f | D local>%.2f | G loss>%.2f" % (epoch,step, d_loss_global, d_loss_local, g_loss))
    

In [ ]:
train(epochs=1)

Start training
epoch> 0 | step> 0 | D global>0.53 | D local>0.53 | G loss>1.97
epoch> 0 | step> 2 | D global>0.51 | D local>0.52 | G loss>1.90
epoch> 0 | step> 4 | D global>0.49 | D local>0.51 | G loss>1.75
epoch> 0 | step> 6 | D global>0.76 | D local>0.83 | G loss>0.71
epoch> 0 | step> 8 | D global>1.27 | D local>0.49 | G loss>0.01
epoch> 0 | step> 10 | D global>1.49 | D local>0.48 | G loss>0.27
epoch> 0 | step> 12 | D global>0.44 | D local>0.50 | G loss>23.72
epoch> 0 | step> 14 | D global>0.50 | D local>0.45 | G loss>3.03
epoch> 0 | step> 16 | D global>0.44 | D local>0.44 | G loss>1.83
epoch> 0 | step> 18 | D global>0.43 | D local>0.42 | G loss>1.69
epoch> 0 | step> 20 | D global>0.42 | D local>0.41 | G loss>1.57
epoch> 0 | step> 22 | D global>0.39 | D local>0.38 | G loss>1.45
epoch> 1 | step> 0 | D global>0.37 | D local>0.36 | G loss>1.31
epoch> 1 | step> 2 | D global>0.14 | D local>0.14 | G loss>1.17
epoch> 1 | step> 4 | D global>0.31 | D local>0.30 | G loss>0.99
epoch> 1 | step> 

In [ ]:
path = "/content/drive/MyDrive/DATA/eval15/"
# load dataset (low light image) - Monet paintings
evalH_all = load_images(path + 'high/')
evalH_all  = evalH_all .astype('float32') / 255.
print('Loaded dataH: ', dataH_all.shape)
# load dataset (low light image) - Monet paintings
evalL_all  = load_images(path + 'low/')
evalL_all  = evalL_all .astype('float32') / 255.
print('Loaded dataL: ', dataL_all.shape)
generator.evaluate(evalL_all, evalH_all)

generator.save('/content/drive/MyDrive/Colab Notebooks/MAGAN-F-200.h5')

In [ ]:
# from keras.models import load_model
# from numpy.random import randn
# model2 = load_model('/content/drive/MyDrive/Colab Notebooks/lle_autoencoder200.h5') #Model trained for 100 epochs

path = '/content/drive/MyDrive/DATA/test/'
# load dataset (low light image)
x_test = load_images(path + 'low/')

results_img = generator.predict(x_test)
results_img.shape

# create figure
fig = plt.figure(figsize=(10, 7))
plt.axis("off")
# Adds a subplot at the 1st position
fig.add_subplot(2, 3, 1)
# showing image
plt.imshow(x_test[0])
fig.add_subplot(2, 3, 2)
# showing image
plt.imshow(x_test[1])
fig.add_subplot(2, 3, 3)
# showing image
plt.imshow(x_test[2])

fig.add_subplot(2, 3, 4)
# showing image
plt.imshow(results_img[0])
fig.add_subplot(2, 3, 5)
# showing image
plt.imshow(results_img[1])
fig.add_subplot(2, 3, 6)
# showing image
plt.imshow(results_img[2])
plt.show()